In [120]:
import requests
from urllib.parse import urlparse
import json
import re
import os
import pandas as pd
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from tqdm import tqdm

In [ ]:
# Сбор ссылок по каждой категии, добавление ссылок пагинации поучение всех сылок страниц с книгами. Извлечение ссылок по каждой гниге.
# Добавление в датасет данных. 

In [121]:
url = 'https://books.toscrape.com/'
user = UserAgent()
headers = {
    "User-Agent":user.random,
}
params = {}
session = requests.session()
response = session.get(url=url, params=params, headers=headers)

In [122]:
def get_categories_from_response(response):
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, features='html.parser')
    else:
        print(response.status_code)
    url_categories = []
    result = soup.find_all(name="a", attrs={"href":re.compile("^catalogue/category/books/")})
    for i in result:
        url_categories.append(i.get("href"))
    url_categories_links = []
    for i in url_categories:
        url_categories_links.append(f'{url}{i}')
    return url_categories_links

In [123]:
url_categories_links = get_categories_from_response(response)

In [124]:
def get_url_from_row(url):
    result = []
    response = session.get(url=url, params=params, headers=headers)
    soup = BeautifulSoup(response.text, features='html.parser')
    data_page_article = soup.find(name="ol", attrs={"class":"row"}).find_all(name="h3")
    for card in data_page_article:
        result.append("/".join(url.split("/")[:4]+(card.find("a").get('href')).split("/")[-2:]))
    return result

In [125]:
def urls_from_paginator(urls):
        url_from_paginator = []
        for url in tqdm(urls):
                while url != None:
                        response = session.get(url=url, params=params, headers=headers)
                        soup = BeautifulSoup(response.text, features='html.parser')
                        if soup.find(name="li", attrs={"class":"next"}) is not None:
                                next_lin="/".join(url.split("/")[:-1]+[soup.find(name="li", attrs={"class":"next"}).findChild("a").get("href")])
                                url_from_paginator.append(next_lin)
                                url = next_lin
                        else:
                                url = None
        result = url_from_paginator
        return result

In [126]:
urls_paginator = urls_from_paginator(url_categories_links)

100%|██████████| 50/50 [00:19<00:00,  2.62it/s]


In [127]:
print(len(urls_paginator))
print(len(url_categories_links))
url_categories_links.extend(urls_paginator)
print(len(url_categories_links))

30
50
80


In [128]:
def all_books_links(urls):
    finall_all_links = []
    for url in tqdm(url_categories_links):
        link_from_row = get_url_from_row(url)
        finall_all_links.extend(link_from_row)
    return finall_all_links


In [129]:
boks_links = all_books_links(url_categories_links)

100%|██████████| 80/80 [00:18<00:00,  4.35it/s]


In [130]:
columns = ['name','price', 'count', 'description']
result_data = pd.DataFrame(columns = columns)

In [ ]:
def data_extraxt(url):
    result = {}
    response = session.get(url=url, params=params, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, features='html.parser')
    else:
        print(response.status_code)

    card = soup.find(name="div", attrs={"class":"product_main"})
    if card:
        result['name'] = card.find(name="h1").text
        price = card.find(name="p", attrs={"class":"price_color"}).text
        result['price'] = float(price[2:])
        count = card.find(name="p", attrs={"class":"instock availability"}).text
        pattern = r'(\d{1,9})'
        count = int(re.search(pattern,count).group())
        result['count'] = count
    descriptions = soup.find(name="article", attrs={"class":"product_page"}).find(name="p", attrs={'class': False, 'id': False})
    if descriptions:
        result['description'] = descriptions.text
    return result

In [132]:
for index, link in enumerate(tqdm(boks_links)):
    data_page = data_extraxt(link)
    result_data.loc[index] = data_page.get('name'), data_page.get('price'), data_page.get('count'), data_page.get('description')
   

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [02:37<00:00,  6.36it/s]


In [133]:
result_data.head()

,name,price,count,description
0,It's Only the Himalayas,45.17,19,"âWherever you go, whatever you do, just . . ..."
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43,15,Acclaimed travel writer Rick Antonson sets his...
2,See America: A Celebration of Our National Par...,48.87,14,To coincide with the 2016 centennial anniversa...
3,Vagabonding: An Uncommon Guide to the Art of L...,36.94,8,With a new foreword by Tim Ferriss â¢Thereâ...
4,Under the Tuscan Sun,37.33,7,A CLASSIC FROM THE BESTSELLING AUTHOR OF UNDER...


In [134]:
result_data.to_json('file.json', orient = 'split', compression = 'infer', index = 'true')

In [136]:
result_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         1000 non-null   object 
 1   price        1000 non-null   float64
 2   count        1000 non-null   int64  
 3   description  998 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 71.4+ KB


In [145]:
result_data.describe()

,price,count
count,1000.00000,1000.000000
mean,35.07035,8.585000
std,14.44669,5.654622
min,10.00000,1.000000
25%,22.10750,3.000000
50%,35.98000,7.000000
75%,47.45750,14.000000
max,59.99000,22.000000


In [142]:
result_data.isna().sum()

name           0
price          0
count          0
description    2
dtype: int64

In [150]:
result_data[result_data['name'].duplicated()]

,name,price,count,description
296,The Star-Touched Queen,32.3,12,Fate and fortune. Power and passion. What does...


In [ ]:
result_data[result_data['name']=='The Star-Touched Queen'] #разные экземпляры

,name,price,count,description
289,The Star-Touched Queen,46.02,14,Fate and fortune. Power and passion. What does...
296,The Star-Touched Queen,32.30,12,Fate and fortune. Power and passion. What does...
